In [1]:
import numpy as np
import scipy.stats
import matplotlib
import matplotlib.pyplot as plt
from elfi.examples.gauss import get_model

%matplotlib inline
%precision 2

import logging
logging.basicConfig(level=logging.INFO)

# Set an arbitrary global seed to keep the randomly generated quantities the same
seed = 1
np.random.seed(seed)

import elfi
N = 1000

### 1-D Gaussian model

In [2]:
true_mu_1d = [4]
cov_matrix_1d = [1]

In [3]:
m_1d = get_model(true_params=true_mu_1d, nd_mean=True, cov_matrix=cov_matrix_1d, seed_obs=seed)

In [4]:
m_1d.parameter_names

['mu_0']

In [5]:
rej_1d = elfi.Rejection(m_1d['d'], batch_size=N, seed=seed)

In [6]:
bounds_1d = {'mu_0': (-2, 2)}

In [7]:
gp_1d = elfi.GPyRegression(m_1d.parameter_names, bounds_1d)

In [8]:
prior_1d = elfi.methods.utils.ModelPrior(m_1d)

In [9]:
acq_1d = elfi.methods.bo.acquisition.RandMaxVar(model=gp_1d, prior=prior_1d)

In [10]:
log_1d = elfi.Operation(np.log, m_1d['d'])

In [11]:
initial_evidence = 10
update_interval = 1
n_rejection_samples = 2 * initial_evidence

In [12]:
rej_result_1d = rej_1d.sample(n_samples=n_rejection_samples, quantile = 1)

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [13]:
bolfi_1d = elfi.BOLFI(log_1d, batch_size=1, initial_evidence=rej_result_1d.outputs, 
                   update_interval=update_interval, bounds=bounds_1d, seed=scipy.random.seed(), 
                   async=True, target_model=gp_1d, acquisition_method=acq_1d, acq_noise_var=[0.1], 
                   batches_per_acquisition=1)

KeyError: 'log_1d'

### 2-D Gaussian model

In [ ]:
true_mu_2d = [2, 3]
cov_matrix_2d = [3, 4]

In [ ]:
m_2d = get_model(true_params=true_mu_2d, nd_mean=True, cov_matrix=cov_matrix_2d, seed_obs=seed)

In [ ]:
m_2d.parameter_names

In [ ]:
bounds_2d = {'mu_0': (-2, 2), 'mu_1': (-1, 1)}

In [ ]:
gp_2d = elfi.GPyRegression(m_2d.parameter_names, bounds_2d)

In [ ]:
prior_2d = elfi.methods.utils.ModelPrior(m_2d)

In [ ]:
acq_2d = elfi.methods.bo.acquisition.RandMaxVar(model=gp_2d, prior=prior_2d)

In [ ]:
log_2d = elfi.Operation(np.log, m_2d['d'])

In [ ]:
rej_2d = elfi.Rejection(m_2d['d'], batch_size = 1)

In [ ]:
rej_result_2d = rej.sample(n_samples=n_rejection_samples,quantile = 1)

In [ ]:
bolfi_2d = elfi.BOLFI(log_2d, batch_size=1, initial_evidence=rej_result_2d.outputs, 
                   update_interval=update_interval, bounds=bounds_2d, seed=scipy.random.seed(), 
                   async=True, target_model=gp_2d, acquisition_method=acq_2d, acq_noise_var=[0.1, 0.01], 
                   batches_per_acquisition=1)

### 3-D Gaussian model

In [ ]:
true_mu_3d = [2, 5, 6]
cov_matrix_3d = [3, 4, 7]

In [ ]:
m_3d = get_model(true_params=true_mu_3d, nd_mean=True, cov_matrix=cov_matrix_3d, seed_obs=seed)

In [ ]:
m_3d.parameter_names

In [ ]:
rej_3d = elfi.Rejection(m_3d['d'], batch_size=N, seed=seed)

In [ ]:
bounds_3d = {'mu_0': (-2, 2), 'mu_1': (-1, 1), 'mu_2': (-3, 3)}

In [ ]:
gp_3d = elfi.GPyRegression(m_3d.parameter_names, bounds_3d)

In [ ]:
prior_3d = elfi.methods.utils.ModelPrior(m_3d)

In [ ]:
acq_3d = elfi.methods.bo.acquisition.RandMaxVar(model=gp_3d, prior=prior_3d)

In [ ]:
log_3d = elfi.Operation(np.log, m_3d['d'])

In [ ]:
rej_result_3d = rej.sample(n_samples=n_rejection_samples,quantile = 1)

In [ ]:
bolfi_3d = elfi.BOLFI(log_3d, batch_size=1, initial_evidence=rej_result_3d.outputs, 
                   update_interval=update_interval, bounds=bounds_3d, seed=scipy.random.seed(), 
                   async=True, target_model=gp_3d, acquisition_method=acq_3d, acq_noise_var=[0.1, 0.01, 0.1], 
                   batches_per_acquisition=1)